# Step 1: Match voter file counts to ACS CVAP Counts

First, establish the "count" of registered voters in a given area.  I do this by narrowing to active voters only and those that are currently living at their address (per TargetSmart data). The example here is LD 26 in Arizona. 

In [1]:
#import dependencies
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
# Load .csv files- voter file data and acs data
# Unzip BlockGr file first
vf_data = "Resources/az_ld_26/az_ld_26.csv"
statewide_data = "Resources/az_statewide/az_statewide.csv"
acs_data = "Resources/BlockGr/BlockGr.csv"

# convert .csv data to Pandas dataframe
az_df = pd.read_csv(vf_data, encoding = "ISO-8859-1", dtype={'reg_census_id': object})
acs_cvap = pd.read_csv(acs_data, encoding = "ISO-8859-1")
state_df = pd.read_csv(statewide_data, encoding = "ISO-8859-1", dtype={'reg_census_id': object})

In [3]:
az_df_count = az_df['voterbase_id'].count()
az_df_count

122387

In [4]:
#Building our list of current voters. First, narrow to just active voters
reg_voters = az_df.loc[az_df["vf_voter_status"] == "Active", :]
reg_voters_count = reg_voters['voterbase_id'].count()
reg_voters_count

97917

In [5]:
#Then, narrow to just those voters with a street level geocode
geo_voters = reg_voters.loc[reg_voters["reg_level"] == "Street", :]
geo_voters_count = geo_voters['voterbase_id'].count()
geo_voters_count

97673

In [6]:
#Then, remove voters who are no longer living at their current address (according to targetsmart data)
#This will be the list the count of voters we compare to ACS CVAP data
vf_df = geo_voters.loc[geo_voters["voterbase_mover_status"] == "Resides at Current Address", :]
vf_count = vf_df['voterbase_id'].count()
vf_count

82219

In [7]:
# convert reg_census_id to string, create new census block group column and state fips column from reg_census_id
vf_df.reg_census_id = vf_df.reg_census_id.astype(str)
vf_df['block_group'] = vf_df.reg_census_id.str[:12]
vf_df['state_fips'] = vf_df.reg_census_id.str[:2]

C:\Users\chris.brill\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\chris.brill\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\chris.brill\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWi

In [8]:
# insert check to make sure just records with An Arizona census block are making it through (AZ FIPS code = 04)
az_df = vf_df.loc[vf_df["state_fips"] == "04", :]
az_count = az_df['voterbase_id'].count()
az_count

82219

In [9]:
az_df.head()

,voterbase_id,vf_voter_status,vf_county_name,vf_cd,vf_hd,vf_precinct_name,reg_level,reg_census_id,voterbase_mover_status,tsmr_race,block_group,state_fips
1506,AZ-5102234,Active,MARICOPA,7,26,ELWOOD,Street,040131162042014,Resides at Current Address,Hispanic,040131162042,04
1612,AZ-000000687577,Active,MARICOPA,7,26,ELWOOD,Street,040131162042009,Resides at Current Address,Hispanic,040131162042,04
1613,AZ-4878209,Active,MARICOPA,7,26,ELWOOD,Street,040131162042009,Resides at Current Address,Hispanic,040131162042,04
1614,AZ-5082375,Active,MARICOPA,7,26,ELWOOD,Street,040131162042009,Resides at Current Address,Hispanic,040131162042,04
1615,AZ-6172918,Active,MARICOPA,7,26,ELWOOD,Street,040131162042009,Resides at Current Address,Hispanic,040131162042,04


In [10]:
# Registered voters by census block group
bg_counts = az_df["block_group"].value_counts()
reg_table = pd.DataFrame({"reg_count": bg_counts,
                                 })

In [11]:
# narrow ACS data down to just total CVAP counts, create block group column from geoid column. Narrow to Maricopa County only (Fips: 04013)
acs_cvap = acs_cvap.loc[acs_cvap["lntitle"] == "Total", :]
acs_cvap = acs_cvap.rename(columns={'CVAP_EST': 'CVAP_estimate',
                                   'CVAP_MOE': 'CVAP_margin_of_error'})
acs_cvap['block_group'] = acs_cvap.geoid.str[7:]
acs_cvap['county_fips'] = acs_cvap.block_group.str[:5]
maricopa_cvap = acs_cvap.loc[acs_cvap["county_fips"] == "04013", :]
maricopa_cvap.head()

,GEONAME,lntitle,geoid,lnnumber,CIT_EST,CIT_MOE,CVAP_estimate,CVAP_margin_of_error,block_group,county_fips
56303,"Block Group 1, Census Tract 101.01, Maricopa C...",Total,15000US040130101011,1,1790,319,1480,275,040130101011,04013
56316,"Block Group 2, Census Tract 101.01, Maricopa C...",Total,15000US040130101012,1,1710,494,1380,303,040130101012,04013
56329,"Block Group 3, Census Tract 101.01, Maricopa C...",Total,15000US040130101013,1,1280,236,1280,236,040130101013,04013
56342,"Block Group 1, Census Tract 101.02, Maricopa C...",Total,15000US040130101021,1,975,392,945,386,040130101021,04013
56355,"Block Group 2, Census Tract 101.02, Maricopa C...",Total,15000US040130101022,1,1390,327,1360,323,040130101022,04013


In [12]:
cvap_final = maricopa_cvap.drop(columns=['GEONAME', 'lntitle', 'geoid', 'lnnumber', 'CIT_EST', 'CIT_MOE' ])
cvap_final = cvap_final[["county_fips","block_group","CVAP_estimate","CVAP_margin_of_error"]]
cvap_final.set_index('block_group')
cvap_final.head()

,county_fips,block_group,CVAP_estimate,CVAP_margin_of_error
56303,04013,040130101011,1480,275
56316,04013,040130101012,1380,303
56329,04013,040130101013,1280,236
56342,04013,040130101021,945,386
56355,04013,040130101022,1360,323


In [13]:
# Combine ACS CVAP count with voter file registration count by census block group
merged_data = pd.merge(reg_table, cvap_final, left_index=True, right_on='block_group')
merged_data = merged_data[["county_fips", "block_group", "CVAP_estimate", "CVAP_margin_of_error", "reg_count"]]
merged_data.head(100)

,county_fips,block_group,CVAP_estimate,CVAP_margin_of_error,reg_count
88790,04013,040139413004,1555,343,1231
79469,04013,040134222031,2190,263,1129
76245,04013,040133195004,1745,367,1114
76024,04013,040133193001,1910,312,1015
79443,04013,040134221071,2005,321,956
75725,04013,040133189001,2055,611,948
76180,04013,040133194041,1480,218,945
78585,04013,040134212012,1810,280,936
75582,04013,040133184002,1075,305,925
79066,04013,040134218022,1415,260,917


In [14]:
active_voters = state_df.loc[state_df["vf_voter_status"] == "Active", :]
state_geo_voters = active_voters.loc[active_voters["reg_level"] == "Street", :]
state_final = state_geo_voters.loc[state_geo_voters["voterbase_mover_status"] == "Resides at Current Address", :]
state_final.reg_census_id = state_final.reg_census_id.astype(str)
state_final['block_group'] = state_final.reg_census_id.str[:12]
state_final['state_fips']  = state_final.reg_census_id.str[:2]
state_final.head()

C:\Users\chris.brill\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\chris.brill\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,voterbase_id,vf_voter_status,vf_county_name,vf_hd,reg_level,reg_census_id,voterbase_mover_status,tsmr_race,block_group,state_fips
4887,AZ-000000562269,Active,MARICOPA,27.0,Street,040131149001045,Resides at Current Address,Hispanic,040131149001,04
4889,AZ-6734255,Active,MARICOPA,27.0,Street,040131142001057,Resides at Current Address,Hispanic,040131142001,04
4891,AZ-000000554541,Active,MARICOPA,27.0,Street,040131142001053,Resides at Current Address,Hispanic,040131142001,04
4893,AZ-6941812,Active,MARICOPA,24.0,Street,040131131001008,Resides at Current Address,Caucasian,040131131001,04
4894,AZ-5340916,Active,MARICOPA,24.0,Street,040131131001001,Resides at Current Address,Caucasian,040131131001,04


In [15]:
state_bg_counts = state_final["block_group"].value_counts()
state_reg_table = pd.DataFrame({"state_reg_count": state_bg_counts,
                                 })
state_reg_table.head()

,state_reg_count
040250019004,7500
040190046381,7038
040138160001,5762
040190043291,5417
040190040641,5405


In [16]:
merged_state_counts = pd.merge(merged_data, state_reg_table, right_index=True, left_on='block_group')
merged_state_counts = merged_state_counts.dropna(how='any')
merged_state_counts.head()

,county_fips,block_group,CVAP_estimate,CVAP_margin_of_error,reg_count,state_reg_count
88790,04013,040139413004,1555,343,1231,1233
79469,04013,040134222031,2190,263,1129,1135
76245,04013,040133195004,1745,367,1114,1116
76024,04013,040133193001,1910,312,1015,1022
79443,04013,040134221071,2005,321,956,957


In [22]:
#calculate the percent of each block group that is registered
merged_state_counts['percent'] = merged_state_counts.reg_count / merged_state_counts.state_reg_count
merged_state_counts['adj_CVAP_estimate'] = merged_state_counts.percent * merged_state_counts.CVAP_estimate
merged_state_counts.head()

,county_fips,block_group,CVAP_estimate,CVAP_margin_of_error,reg_count,state_reg_count,percent,adj_CVAP_estimate,reg_potential
88790,04013,040139413004,1555,343,1231,1233,0.998378,1552.477697,321.477697
79469,04013,040134222031,2190,263,1129,1135,0.994714,2178.422907,1049.422907
76245,04013,040133195004,1745,367,1114,1116,0.998208,1741.872760,627.872760
76024,04013,040133193001,1910,312,1015,1022,0.993151,1896.917808,881.917808
79443,04013,040134221071,2005,321,956,957,0.998955,2002.904911,1046.904911


In [43]:
adj_CVAP_estimate = merged_state_counts['adj_CVAP_estimate'].astype(float).astype(int)
merged_state_counts['reg_potential'] = merged_state_counts.adj_CVAP_estimate - merged_state_counts.reg_count
merged_state_counts['percent_reg'] = merged_state_counts.reg_count / merged_state_counts.adj_CVAP_estimate
merged_state_counts.head()

,county_fips,block_group,CVAP_estimate,CVAP_margin_of_error,reg_count,state_reg_count,percent,adj_CVAP_estimate,reg_potential,percent_reg
88790,04013,040139413004,1555,343,1231,1233,0.998378,1552.477697,321.477697,0.792926
79469,04013,040134222031,2190,263,1129,1135,0.994714,2178.422907,1049.422907,0.518265
76245,04013,040133195004,1745,367,1114,1116,0.998208,1741.872760,627.872760,0.639542
76024,04013,040133193001,1910,312,1015,1022,0.993151,1896.917808,881.917808,0.535079
79443,04013,040134221071,2005,321,956,957,0.998955,2002.904911,1046.904911,0.477307


In [50]:
#sort data on registration potential.  Top census block groups for voter registration
merged_state_counts.sort_values("reg_potential", inplace=True, ascending=True)
merged_state_counts.head(25)

cm = sns.light_palette("purple", as_cmap=True)

final_table= merged_state_counts.style.background_gradient(cmap=cm)

final_table

,county_fips,block_group,CVAP_estimate,CVAP_margin_of_error,reg_count,state_reg_count,percent,adj_CVAP_estimate,reg_potential,percent_reg
76505,04013,040133198006,565,191,642,646,0.993808,561.502,-80.4985,1.14336
80002,04013,040134223013,800,281,874,876,0.997717,798.174,-75.8265,1.095
76141,04013,040133194032,810,174,876,876,1,810,-66,1.08148
79209,04013,040134220022,385,155,388,390,0.994872,383.026,-4.97436,1.01299
75270,04013,040132177001,2100,341,1,1417,0.000705716,1.482,0.482004,0.674762
73515,04013,040132168163,1745,466,1,970,0.00103093,1.79897,0.798969,0.555874
56342,04013,040130101021,945,386,2,539,0.00371058,3.50649,1.50649,0.57037
71825,04013,040131162051,735,142,7,578,0.0121107,8.90138,1.90138,0.786395
76466,04013,040133198003,360,163,354,357,0.991597,356.975,2.97479,0.991667
71851,04013,040131162053,1370,279,2,521,0.00383877,5.25912,3.25912,0.380292


In [51]:
# Export file as a CSV
merged_state_counts.to_csv("Output/ld26_reg_analysis.csv", index=False, header=True)